## import library

In [2]:
import numpy as np
import pandas as pd
import itertools
from konlpy.tag import Okt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from collections import Counter
from PIL import Image
import pickle

In [3]:
class dfchange:
    def __init__(self,DF):
        
        
        self.DF = DF.reset_index(drop=True)  # 데이터프레임 받기.
            
     
    #온도측정
    def get_temp(self):
        
        df = self.DF
        #비율
        
        count_0 = len(df[df.emotion == '부정'])
        count_1 = len(df[df.emotion == '긍정'])
        count_2 = len(df[df.emotion == '중립'])
        
        new_sum = count_0 + count_1
        
        percent_0 = count_0/new_sum
        percent_1 = count_1/new_sum
        
        
        
        if percent_0 != 0:
            value = 30 * percent_0 + 42 * percent_1
        else:
            value = 50
        
        return value
    
    
    #긍정 데이터프레임 뽑기
    def get_1(self):
        
        DF = self.DF
        DF.reset_index(inplace=True,drop=True)
        index_1=DF[(DF.emotion=='긍정')].index
        DF_1=DF.loc[index_1]
        return DF_1

    
    #부정 데이터프레임 뽑기
    def get_0(self):
        
        DF = self.DF
        DF.reset_index(inplace=True,drop=True)
        index_0=DF[(DF.emotion=='부정')].index
        DF_0=DF.loc[index_0]
        return DF_0
    
    #중립 데이터프레임 뽑기
    def get_2(self):
        DF = self.DF
        DF.reset_index(inplace=True,drop=True)
        index_2=DF[(DF.emotion=='중립')].index
        DF_2=DF.loc[index_2]
        return DF_2
    
    #측정된 온도 데이터프레임 뽑기
    def get_this(self):
        DF=self.DF
        percent_0=len(DF[DF.emotion=='부정'])/len(DF)
        percent_1=len(DF[DF.emotion=='긍정'])/len(DF)
        max_percent=max(percent_0,percent_1)   
        
        if percent_0 > percent_1 :
            v = '부정'

        elif percent_1> percent_0:
            v = '긍정'
            
        else :
            v = '중립'

        
        return v
    
    #온도 측정에 따른 그림 출력
    
    
    def bar(self,file,key):
        DataFrame = self.DF
        plt.show()
        
        labels = ['Negative', 'Positive', 'Neutral']
        values = [len(DataFrame[DataFrame['emotion']=="부정"]), len(DataFrame[DataFrame['emotion']=="긍정"]), len(DataFrame[DataFrame['emotion']=="중립"])]
        colors = ['#ffc000', '#ff9999','#8fd9b6']
        plt.bar(labels, values, color=colors, edgecolor='black')
        plt.xticks(labels)
        #plt.title("Today's emotional count")
        #plt.show()
            
        # save img and stop function runnning.
        if file == 'positive':
            return plt.savefig(f'daily_news/positive/bar_graph.PNG')
        elif file == 'negative':
            return plt.savefig(f'daily_news/negative/bar_graph.PNG')
        else:
            return plt.savefig(f'sector/{key}/{file}/bar_graph.PNG')
            
            
    def ratio(self,file,key):
        DataFrame = self.DF
        plt.show()
        
        ratio_lab0 = len(DataFrame[DataFrame['emotion']=="부정"]) / len(DataFrame) * 100
        ratio_lab1 = len(DataFrame[DataFrame['emotion']=="긍정"]) / len(DataFrame) * 100
        ratio_lab2 = len(DataFrame[DataFrame['emotion']=="중립"]) / len(DataFrame) * 100
        
        ratio = [ratio_lab0,ratio_lab1,ratio_lab2]
        labels = ['Negative', 'Positive', 'Neutral']
        explode = [0, 0, 0]
        colors = ['#ffc000', '#ff9999','#8fd9b6']
        wedgeprops={'width': 0.7, 'edgecolor': 'w', 'linewidth': 5}
        plt.pie(ratio, labels=labels, autopct='%.1f%%', explode=explode, counterclock=False,shadow=True, colors=colors, wedgeprops=wedgeprops)
        #plt.title("Today's emotional ratio")
        #plt.show()
        
        # save img and stop function runnning.
        if file == 'positive':
            return plt.savefig(f'daily_news/positive/ratio_graph.PNG')
        elif file == 'negative':
            return plt.savefig(f'daily_news/negative/ratio_graph.PNG')
        else:
            return plt.savefig(f'sector/{key}/{file}/ratio_graph.PNG')
            

In [ ]:
class Keybert:
    def __init__(self,DF):
          self.DF=DF.reset_index(drop=True)  # 데이터프레임 받기.
            
    
            
    def get_Keyword(self,Mode='MSS',n=5,nrc=10,diversity=0.2):

        #데이터프레임 문장 합치기
        doc=''
        for i in self.DF.index:
            doc += (self.DF.loc[i,'title'])
            
            
        # 형태소 분석기를 통해 명사만 추출한 문서를 만들기
        
        okt = Okt()  #형태소분석기 
        tokenized_doc = okt.pos(doc)
        tokenized_nouns = ' '.join([word[0] for word in tokenized_doc if word[1] == 'Noun'])
        
        #CountVectorizer를 사용하여 단어를 추출
        
        n_gram_range = (2, 3)
        count = CountVectorizer(ngram_range=n_gram_range).fit([tokenized_nouns])
        candidates = count.get_feature_names()
        
        
        #문서와 문서로부터 추출한 키워드들을 SBERT를 통해서 수치화
        
        model = SentenceTransformer('sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens')
        doc_embedding = model.encode([doc])
        candidate_embeddings = model.encode(candidates)

        if Mode == 'MSS':
            keywords = self.max_sum_sim(doc_embedding=doc_embedding, candidate_embeddings=candidate_embeddings, words=candidates, top_n=n, nr_candidates=nrc)
            return keywords
        elif Mode == 'MMR':
            keywords=self.mmr(doc_embedding=doc_embedding, candidate_embeddings=candidate_embeddings, words=candidates, top_n=n, diversity=diversity)
            return keywords
        else:
       #문서와 가장 유사한 키워드들을 추출

            top_n = n  #상위 n개
            distances = cosine_similarity(doc_embedding, candidate_embeddings)
            keywords = [candidates[index] for index in distances.argsort()[0][-top_n:]]
            return keywords
        
        

    def get_Keyword_c(self,Mode='MSS',n=5,nrc=10,diversity=0.2):
    #데이터프레임 문장 합치기
        doc = ''
        for i in self.DF.index:
            doc += (self.DF.loc[i,'title'])

        # 형태소 분석기를 통해 명사만 추출한 문서를 만들기

        okt = Okt()  #형태소분석기 
        tokenized_doc = okt.pos(doc)
        tokenized_nouns = ' '.join([word[0] for word in tokenized_doc if word[1] == 'Noun'])

        #CountVectorizer를 사용하여 단어를 추출

        n_gram_range = (2, 3)
        count = CountVectorizer(ngram_range=n_gram_range).fit([tokenized_nouns])
        candidates = count.get_feature_names()
        

        #문서와 문서로부터 추출한 키워드들을 SBERT를 통해서 수치화

        model = SentenceTransformer('sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens')
        doc_embedding = model.encode([doc])
        candidate_embeddings = model.encode(candidates)

        if Mode == 'MSS':
            keywords = self.max_sum_sim(doc_embedding=doc_embedding, candidate_embeddings=candidate_embeddings, words=candidates, top_n=n, nr_candidates=nrc)
            return keywords
        elif Mode == 'MMR':
            keywords = self.mmr(doc_embedding=doc_embedding, candidate_embeddings=candidate_embeddings, words=candidates, top_n=n, diversity=diversity)
            return keywords
        else:
       #문서와 가장 유사한 키워드들을 추출
            top_n = n  #상위 n개
            distances = cosine_similarity(doc_embedding, candidate_embeddings)
            keywords = [candidates[index] for index in distances.argsort()[0][-top_n:]]
            return keywords
        
    #워드클라우드 출력   
    def get_wordcloud(self,bc='black',font='malgun'):
        
        #데이터프레임 문장 합치기
        doc = ''
        for i in self.DF.index:
            doc += (self.DF.loc[i,'title'])
        text=doc
        
        okt = Okt()
        nouns = okt.nouns(text) # 명사만 추출
        words = [n for n in nouns if len(n) > 1] # 단어의 길이가 1개인 것은 제외
        c = Counter(words) # 위에서 얻은 words를 처리하여 단어별 빈도수 형태의 딕셔너리 데이터를 구함
        
        # 한글로 워드크라우드 시각화
        #적재시 폰트 malgun이 적용되는가?
        wc = WordCloud(font_path=font, background_color=bc,
                       width=400, height=400, scale=2.0, max_font_size=250)
        gen = wc.generate_from_frequencies(c)
        plt.figure()
        plt.imshow(gen)
        return gen

    def get_wordcloud_c(self,bc='black',font='malgun'):
        
    #데이터프레임 문장 합치기
        doc=''
        for i in self.DF.index:
            doc += (self.DF.loc[i,'title'])
        text=doc

        okt = Okt()
        nouns = okt.nouns(text) # 명사만 추출
        words = [n for n in nouns if len(n) > 1] # 단어의 길이가 1개인 것은 제외
        c = Counter(words) # 위에서 얻은 words를 처리하여 단어별 빈도수 형태의 딕셔너리 데이터를 구함

        # 한글로 워드크라우드 시각화
        #적재시 폰트 malgun이 적용되는가?
        wc = WordCloud(font_path=font, background_color=bc,
                       width=400, height=400, scale=2.0, max_font_size=250)
        gen = wc.generate_from_frequencies(c)
        plt.figure()
        plt.imshow(gen)
        return gen
    
    #데이터프레임 링크 출력
    def get_link(self,emotion):
        DF=self.DF
        DF.reset_index(inplace=True,drop=True)
        emotion_index=DF[(DF.emotion==emotion)].index
        DF=DF.loc[emotion_index]
        li=[]
        li.append(DF.url)
        
        return li
    
        
        
        
        
        
    def max_sum_sim(self,doc_embedding, candidate_embeddings, words, top_n, nr_candidates):
        # 문서와 각 키워드들 간의 유사도
        distances = cosine_similarity(doc_embedding, candidate_embeddings)

        # 각 키워드들 간의 유사도
        distances_candidates = cosine_similarity(candidate_embeddings, 
                                                candidate_embeddings)

        # 코사인 유사도에 기반하여 키워드들 중 상위 top_n개의 단어를 pick.
        words_idx = list(distances.argsort()[0][-nr_candidates:])
        words_vals = [words[index] for index in words_idx]
        distances_candidates = distances_candidates[np.ix_(words_idx, words_idx)]

        # 각 키워드들 중에서 가장 덜 유사한 키워드들간의 조합을 계산
        min_sim = np.inf
        candidate = None
        for combination in itertools.combinations(range(len(words_idx)), top_n):
            sim = sum([distances_candidates[i][j] for i in combination for j in combination if i != j])
            if sim < min_sim:
                candidate = combination
                min_sim = sim

        return [words_vals[idx] for idx in candidate]
        
    def mmr(self,doc_embedding, candidate_embeddings, words, top_n, diversity):

        # 문서와 각 키워드들 간의 유사도가 적혀있는 리스트
        word_doc_similarity = cosine_similarity(candidate_embeddings, doc_embedding)

        # 각 키워드들 간의 유사도
        word_similarity = cosine_similarity(candidate_embeddings)

        # 문서와 가장 높은 유사도를 가진 키워드의 인덱스를 추출.
        # 만약, 2번 문서가 가장 유사도가 높았다면
        # keywords_idx = [2]
        keywords_idx = [np.argmax(word_doc_similarity)]

        # 가장 높은 유사도를 가진 키워드의 인덱스를 제외한 문서의 인덱스들
        # 만약, 2번 문서가 가장 유사도가 높았다면
        # ==> candidates_idx = [0, 1, 3, 4, 5, 6, 7, 8, 9, 10 ... 중략 ...]
        candidates_idx = [i for i in range(len(words)) if i != keywords_idx[0]]

        # 최고의 키워드는 이미 추출했으므로 top_n-1번만큼 아래를 반복.
        # ex) top_n = 5라면, 아래의 loop는 4번 반복됨.
        for _ in range(top_n - 1):
            candidate_similarities = word_doc_similarity[candidates_idx, :]
            target_similarities = np.max(word_similarity[candidates_idx][:, keywords_idx], axis=1)

            # MMR을 계산
            mmr = (1-diversity) * candidate_similarities - diversity * target_similarities.reshape(-1, 1)
            mmr_idx = candidates_idx[np.argmax(mmr)]

            # keywords & candidates를 업데이트
            keywords_idx.append(mmr_idx)
            candidates_idx.remove(mmr_idx)

        return [words[idx] for idx in keywords_idx]
            
    